CAM 0

In [1]:
import numpy as np

import zarr_io
import data_prep
import coarse_registration

from sofima import stitch_rigid

In [2]:
# Preprocess raw tiles
bucket = 'sofima-test-bucket'
downsampling_exp = 2
path_0 = f'tile_X_0001_Y_0000_Z_0000_CH_0405_cam0.zarr/{downsampling_exp}'  
path_1 = f'tile_X_0002_Y_0000_Z_0000_CH_0405_cam0.zarr/{downsampling_exp}'
path_2 = f'tile_X_0003_Y_0000_Z_0000_CH_0405_cam0.zarr/{downsampling_exp}'
path_3 = f'tile_X_0004_Y_0000_Z_0000_CH_0405_cam0.zarr/{downsampling_exp}'
path_4 = f'tile_X_0005_Y_0000_Z_0000_CH_0405_cam0.zarr/{downsampling_exp}'
path_5 = f'tile_X_0006_Y_0000_Z_0000_CH_0405_cam0.zarr/{downsampling_exp}'

tile_0 = zarr_io.open_zarr(bucket, path_0)
tile_1 = zarr_io.open_zarr(bucket, path_1)
tile_2 = zarr_io.open_zarr(bucket, path_2)
tile_3 = zarr_io.open_zarr(bucket, path_3)
tile_4 = zarr_io.open_zarr(bucket, path_4)
tile_5 = zarr_io.open_zarr(bucket, path_5)

# data_prep.preprocess_tiles(tile_volumes = [tile_0, 
#                                            tile_1, 
#                                            tile_2, 
#                                            tile_3, 
#                                            tile_4, 
#                                            tile_5], 
#                            output_names = ['preprocessed_0.zarr', 
#                                            'preprocessed_1.zarr',
#                                            'preprocessed_2.zarr',
#                                            'preprocessed_3.zarr',
#                                            'preprocessed_4.zarr',
#                                            'preprocessed_5.zarr'],
#                            bucket = bucket, 
#                            batch_size = 256)

I0000 00:00:1684793035.533605   12097 gcs_resource.cc:102] Using default AdmissionQueue with limit 32
I0000 00:00:1684793035.552800   12728 google_auth_provider.cc:179] Running on GCE, using service account 895865026362-compute@developer.gserviceaccount.com


In [3]:
# Coarse Registration
# bucket = 'sofima-test-bucket'
# tile_0 = zarr_io.open_zarr(bucket, "preprocessed_0.zarr")
# tile_1 = zarr_io.open_zarr(bucket, "preprocessed_1.zarr")
# tile_2 = zarr_io.open_zarr(bucket, "preprocessed_2.zarr")
# tile_3 = zarr_io.open_zarr(bucket, "preprocessed_3.zarr")
# tile_4 = zarr_io.open_zarr(bucket, "preprocessed_4.zarr")
# tile_5 = zarr_io.open_zarr(bucket, "preprocessed_5.zarr")

tile_layout = np.array([[0], 
                        [1],
                        [2],
                        [3],
                        [4],
                        [5]])
tile_volumes = [tile_0.T[:,:,:,0,0],
                tile_1.T[:,:,:,0,0], 
                tile_2.T[:,:,:,0,0], 
                tile_3.T[:,:,:,0,0], 
                tile_4.T[:,:,:,0,0], 
                tile_5.T[:,:,:,0,0]]  # Place X/Y dimensions in front, must have 3D
cx, cy = coarse_registration.compute_coarse_offsets(tile_layout, tile_volumes)
coarse_mesh = stitch_rigid.optimize_coarse_mesh(cx, cy, mesh_fn=stitch_rigid.elastic_tile_mesh_3d)

# print(f'{cx=}')
print(f'{cy=}')

Top Id: 0, Bottom Id: 1
Top: (0, 0), Bot: (1, 0) [ -1. 287.   4.]
Top Id: 1, Bottom Id: 2
Top: (1, 0), Bot: (2, 0) [ -1. 286.   0.]
Top Id: 2, Bottom Id: 3
Top: (2, 0), Bot: (3, 0) [ -1. 286.   0.]
Top Id: 3, Bottom Id: 4
Top: (3, 0), Bot: (4, 0) [ -1. 284.   0.]
Top Id: 4, Bottom Id: 5
Top: (4, 0), Bot: (5, 0) [ -1. 287.  -2.]
cy=array([[[[ -1.],
         [ -1.],
         [ -1.],
         [ -1.],
         [ -1.],
         [ nan]]],


       [[[287.],
         [286.],
         [286.],
         [284.],
         [287.],
         [ nan]]],


       [[[  4.],
         [  0.],
         [  0.],
         [  0.],
         [ -2.],
         [ nan]]]])


In [6]:
offsets = []
for i in range(5):
    if i == 0:
        offsets.append(np.array(cy)[:, 0, i, 0])
    else:
        offsets.append(offsets[-1] + np.array(cy)[:, 0, i, 0])

offsets

[array([ -1., 286.,   4.]),
 array([ -2., 572.,   4.]),
 array([ -3., 858.,   4.]),
 array([  -4., 1142.,    4.]),
 array([  -5., 1429.,    2.])]

Patch Size: 25
- Top: (0, 0), Bot: (0, 1) [ -8. 287.   7.]
- Top: (0, 1), Bot: (0, 2) [ 24. 351.   2.]
- Top: (0, 2), Bot: (0, 3) [ 24. 292.  13.]
- Top: (0, 3), Bot: (0, 4) [-22. 307.  25.]
- Top: (0, 4), Bot: (0, 5) [ -3. 287.  -2.]

Patch Size: 50
- Top: (0, 0), Bot: (0, 1) [ -1. 286.   4.]
- Top: (0, 1), Bot: (0, 2) [ -1. 286.   0.]
- Top: (0, 2), Bot: (0, 3) [ -1. 286.   0.]
- Top: (0, 3), Bot: (0, 4) [ -1. 284.   0.]
- Top: (0, 4), Bot: (0, 5) [ -1. 287.  -2.]

In [4]:
np.savez_compressed('coarse_results_6.npz', mesh=coarse_mesh, cx=cx, cy=cy)